<a href="https://colab.research.google.com/github/shaficode786/Incubate_assessment/blob/solution/Incubyte_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Notebook contains my solution(Shafi) for the Data Engineering assessment provided by Incubyte. The assessment involves designing a dynamic ETL process that extracts, validates, and loads customer data into country-specific tables based on the provided specifications:



*  All customers related to India will go to Table_India and so on.


*  If the customer is shifted to another country and visits from that country then consider the record with the latest consultant date.



# 1. Create table queries

To ANswer this, We need to create tables dynamically, i.e., one table for each country. The number of tables will be based on the result of the query: (select count(distinct country) from Staging_Customers)

In [ ]:
-- They are two steps involved in this finding unique countries and creating tables based dynamically based on the results as shown below:

--Step 1: Extract unique countries from the staging table
SELECT DISTINCT Country
INTO #TempCountries
FROM Staging_Customers;

-- Step 2: Loop through each country and create dynamic table queries
DECLARE @country VARCHAR(10), @sql NVARCHAR(MAX);

DECLARE country_cursor CURSOR FOR
SELECT Country FROM #TempCountries;

OPEN country_cursor;
FETCH NEXT FROM country_cursor INTO @country;

WHILE @@FETCH_STATUS = 0
BEGIN
    SET @sql = 'CREATE TABLE Table_' + @country + ' (
                    Customer_Name VARCHAR(255),
                    Customer_Id VARCHAR(18),
                    Open_Date DATE,
                    Last_Consulted_Date DATE,
                    Vaccination_Id CHAR(5),
                    Dr_Name CHAR(255),
                    State CHAR(5),
                    Country CHAR(5),
                    DOB DATE,
                    Is_Active CHAR(1)
                );';
    EXEC sp_executesql @sql;
    FETCH NEXT FROM country_cursor INTO @country;
END;

CLOSE country_cursor;
DEALLOCATE country_cursor;
DROP TABLE #TempCountries;


Sample output if executed:  
CREATE TABLE Table_USA (
                    Customer_Name VARCHAR(255),
                    Customer_Id VARCHAR(18),
                    Open_Date DATE,
                    Last_Consulted_Date DATE,
                    Vaccination_Id CHAR(5),
                    Dr_Name CHAR(255),
                    State CHAR(5),
                    Country CHAR(5),
                    DOB DATE,
                    Is_Active CHAR(1)
                );

# Based on condition "If the customer is shifted to another country and visits from that country then consider the record with the latest consultant date." following records are populated as shown below

In [ ]:

DECLARE @insertSQL NVARCHAR(MAX);

DECLARE country_cursor CURSOR FOR
SELECT Country FROM #TempCountries;

OPEN country_cursor;
FETCH NEXT FROM country_cursor INTO @country;

WHILE @@FETCH_STATUS = 0
BEGIN
    SET @insertSQL = 'INSERT INTO Table_' + @country + ' (Customer_Name, Customer_Id, Open_Date,
                     Last_Consulted_Date, Vaccination_Id, Dr_Name, State, Country, DOB, Is_Active)
                     WITH LatestVisits AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY Customer_Id ORDER BY Last_Consulted_Date DESC) AS rn
    FROM Staging_Customers
)
                     SELECT Customer_Name, Customer_Id, Open_Date, Last_Consulted_Date,
                            Vaccination_Id, Dr_Name, State, Country, DOB, Is_Active
                     FROM LatestVisits
                     WHERE rn = 1 AND Country = ''' + @country + ''';';
    EXEC sp_executesql @insertSQL;
    FETCH NEXT FROM country_cursor INTO @country;
END;

CLOSE country_cursor;
DEALLOCATE country_cursor;


# 2. Create the above tables with additional derived columns: age and days since last consulted >30

To answer this, we need to perform two tasks dynamically:


1. Add the columns age and days_since_last_consulted.

2. Update the derived values in these columns after adding them dynamically.

In [ ]:
-- Step 1: Add the columns age and days_since_last_consulted
DECLARE @country VARCHAR(10);
DECLARE @sql NVARCHAR(MAX);

DECLARE country_cursor CURSOR FOR
SELECT DISTINCT Country FROM Staging_Customers;

OPEN country_cursor;
FETCH NEXT FROM country_cursor INTO @country;

WHILE @@FETCH_STATUS = 0
BEGIN
    SET @sql = 'ALTER TABLE Table_' + @country + '
                ADD Age INT,
                    Days_Since_Last_Consult INT;';
    EXEC sp_executesql @sql;

    FETCH NEXT FROM country_cursor INTO @country;
END;

CLOSE country_cursor;
DEALLOCATE country_cursor;


In [ ]:
-- Step 2: Update the derived values in these columns after adding them dynamically
DECLARE @updateSQL NVARCHAR(MAX);

DECLARE country_cursor CURSOR FOR
SELECT DISTINCT Country FROM Staging_Customers;

OPEN country_cursor;
FETCH NEXT FROM country_cursor INTO @country;

WHILE @@FETCH_STATUS = 0
BEGIN
    -- Generate dynamic UPDATE statement to populate derived columns
    SET @updateSQL = 'UPDATE Table_' + @country + '
                      SET Age = DATEDIFF(YEAR, DOB, GETDATE()),
                          Days_Since_Last_Consult = DATEDIFF(DAY, Last_Consulted_Date, GETDATE())
                      WHERE Last_Consulted_Date IS NOT NULL;';

    -- Execute the dynamic SQL
    EXEC sp_executesql @updateSQL;

    FETCH NEXT FROM country_cursor INTO @country;
END;

CLOSE country_cursor;
DEALLOCATE country_cursor;


# 3. Create necessary validations.

The validation checks we need to perform for this task are:



---



1)Mandatory fields: Customer_Name, Customer_Id, and Open_Date cannot be NULL.

2)Date fields: Validated for correct format using TRY_CAST.

3)Customer_Id: Must be alphanumeric.

4)Is_Active: Can only contain 'A' or 'I'.

5)check for duplicate records in the tables

In [ ]:
-- 1)Mandatory fields validation
SELECT *
FROM Staging_Customers
WHERE Customer_Name IS NULL
   OR Customer_Id IS NULL
   OR Open_Date IS NULL;


In [ ]:
-- 2)date fields format validation using try_cast
SELECT *
FROM Staging_Customers
WHERE TRY_CAST(Open_Date AS DATE) IS NULL
   OR (Last_Consulted_Date IS NOT NULL AND TRY_CAST(Last_Consulted_Date AS DATE) IS NULL)
   OR TRY_CAST(DOB AS DATE) IS NULL;


In [ ]:
--3)Customer_Id: Must be alphanumeric.
SELECT *
FROM Staging_Customers
WHERE Customer_Id LIKE '%[^a-zA-Z0-9]%';


In [ ]:
--4)Is_Active: Can only contain 'A' or 'I'.
SELECT *
FROM Staging_Customers
WHERE Is_Active NOT IN ('A', 'I');


In [ ]:
--5) check for duplicate records in the tables
SELECT Customer_Id, COUNT(*) AS DuplicateCount
FROM table_usa
GROUP BY Customer_Id
HAVING COUNT(*) > 1;